In [1]:
import pandas as pd
import numpy as np
from helper_functions import Coinmetrics

In [ ]:
if __name__ == "__main__":
    # set args
    CM_API_FP = '../../admin/coinmetrics.txt'
    BASE_URL = 'https://api.coinmetrics.io/v4/'
    PANEL_FP = '../data/raw/coinmetrics_panel_initial.pkl'
    TARGET_US_EXCHANGES = ['binance.us',  'bitstamp', 'coinbase', 'crypto.com', 'ftx.us', 
        'gemini', 'kraken', 'kucoin']
    TARGET_METRICS = ['SplyAct1yr', 'SplyActEver', 'SplyCur', 'SplyFF', 
                      'CapMrktCurUSD', 'CapMrktEstUSD', 'CapMrktFFUSD', 'CapRealUSD']
    TARGET_FREQ = '1d'
    STUDY_START = '2016-07-01'
    STUDY_END = '2023-01-03'
    
    # import api key
    with open(CM_API_FP) as f:
        API_KEY = f.readlines()
        API_KEY = API_KEY[0].strip()
    BASE_PARAMS = {'api_key': API_KEY}

    # pull meta data on target exchanges
    exchanges_df = Coinmetrics.pullExchangeInfo(BASE_URL, BASE_PARAMS, TARGET_US_EXCHANGES)

    # pull meta data on coinmetrics assets
    assets_df = Coinmetrics.pullAssetInfo(BASE_URL, BASE_PARAMS)

    # pull meta data markets and subset down to target markets
    markets_df = Coinmetrics.pullAndFormRelevantMarkets(exchanges_df, assets_df, BASE_URL, BASE_PARAMS)

    # pull usdt and usdc exchange rates
    usd_df = Coinmetrics.pullAssetMetrics(BASE_URL, BASE_PARAMS, assets_df, STUDY_START, STUDY_END, 
                                          TARGET_FREQ, ['usdc', 'usdt'], ['ReferenceRateUSD'])
    usd_df = Coinmetrics.cleanUSDPrices(usd_df, TARGET_FREQ)

    # pull ohlcv data
    panel_df = Coinmetrics.pullOHLCV(BASE_URL, BASE_PARAMS, STUDY_START, STUDY_END, TARGET_FREQ, markets_df)
    panel_df = Coinmetrics.cleanOHLCV(panel_df, TARGET_FREQ, markets_df, usd_df)

    # pull mcap data
    asset_universe = list(np.unique(panel_df.asset.values))
    mcap_df = Coinmetrics.pullAssetMetrics(BASE_URL, BASE_PARAMS, assets_df, STUDY_START, STUDY_END, 
                                           TARGET_FREQ, asset_universe, TARGET_METRICS)
    mcap_df = Coinmetrics.cleanPanel(mcap_df)
    
    # merge and save the panel
    panel_df = panel_df.merge(mcap_df, 
                            on=['asset', 'date'],
                            how='outer',
                            validate='one_to_one')
    panel_df = panel_df.sort_values(by=['date', 'asset'], ignore_index=True)
    panel_df.to_pickle(PANEL_FP)